# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [5]:
import pandas as pd

## imports for Python, Pandas

In [6]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [7]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [8]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [9]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [10]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

[{'_id': {'$oid': '52b213b38594d8a2be17c780'},
  'approvalfy': 1999,
  'board_approval_month': 'November',
  'boardapprovaldate': '2013-11-12T00:00:00Z',
  'borrower': 'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  'closingdate': '2018-07-07T00:00:00Z',
  'country_namecode': 'Federal Democratic Republic of Ethiopia!$!ET',
  'countrycode': 'ET',
  'countryname': 'Federal Democratic Republic of Ethiopia',
  'countryshortname': 'Ethiopia',
  'docty': 'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  'envassesmentcategorycode': 'C',
  'grantamt': 0,
  'ibrdcommamt': 0,
  'id': 'P129828',
  'idacommamt': 130000000,
  'impagency': 'MINISTRY OF EDUCATION',
  'lendinginstr': 'Investment Project Financing',
  'lendinginstrtype': 'IN',
  'lendprojectcost': 550000000,
  'majorsector_percent': [{'Name': 'Education', 'Percent': 46},
   {'Name': 'Education', 'Percent': 26},
   {'Name': 'Public Administration, Law, and Justice', 'Percent': 16},
   {'Name': 'Educatio

In [11]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

# 1: Find 10 countries with the most projects

In [12]:
#import necessary package(s)
import pandas as pd
import numpy as np

In [13]:
# Load in the json data as a dataframe for ease of use, select columns for use
projects_json_df = pd.read_json('data/world_bank_projects.json')
projects = projects_json_df[['id','countryshortname','mjtheme_namecode']]

In [14]:
#Find the country totals
projects_count = projects.groupby('countryshortname').count().reset_index()

In [15]:
#Clean for ease of reading, display top 10
projects_top = projects_count[['countryshortname','id']].sort_values('id',ascending=False)
projects_top.columns = ['Country Name', 'Number of Projects']
print(projects_top.head(10))
projects_top = projects_top.head(10)

           Country Name  Number of Projects
23                China                  19
45            Indonesia                  19
112             Vietnam                  17
44                India                  16
115  Yemen, Republic of                  13
73                Nepal                  12
8            Bangladesh                  12
69              Morocco                  12
70           Mozambique                  11
1                Africa                  11


# 2. Find top 10 Major Project Themes

In [16]:
#Pull every code and name instance in the mjtheme_namecode column
projects_mjtheme = projects['mjtheme_namecode']
projects_mjtheme_name=[]
for row in projects_mjtheme :
    for entry in row :
        projects_mjtheme_name.append(entry)
projects_mjtheme_name = pd.DataFrame(projects_mjtheme_name)
print(projects_mjtheme_name)

     code                                          name
0       8                             Human development
1      11                                              
2       1                           Economic management
3       6         Social protection and risk management
4       5                         Trade and integration
5       2                      Public sector governance
6      11  Environment and natural resources management
7       6         Social protection and risk management
8       7                   Social dev/gender/inclusion
9       7                   Social dev/gender/inclusion
10      5                         Trade and integration
11      4      Financial and private sector development
12      6         Social protection and risk management
13      6                                              
14      2                      Public sector governance
15      4      Financial and private sector development
16     11  Environment and natural resources man

In [17]:
#Get a count and sort the order of the types of projects
projects_mjtheme_count = projects_mjtheme_name.groupby('code').count().reset_index()
projects_mjtheme_count.columns = ['Code','Count of Each Code']
projects_mjtheme_count = projects_mjtheme_count.sort_values('Count of Each Code', ascending = False)
print(projects_mjtheme_count.head(10))
#Note: Only one code is dropped, the code 3, which represents Rule of Law

   Code  Count of Each Code
2    11                 250
1    10                 216
9     8                 210
3     2                 199
7     6                 168
5     4                 146
8     7                 130
6     5                  77
10    9                  50
0     1                  38


# 3. Replace Missing Names

Approach 1: Replace the missing names only in the mjtheme dataframe

In [18]:
#Using projects_mjtheme_name from above, replace the empty strings with NaN
projects_mjtheme_missing = projects_mjtheme_name.replace('',np.nan)
#Sort the dataframe
projects_mjtheme_missing = projects_mjtheme_missing.sort_values('code')
#Forward fill the dataframe
projects_mjtheme_missing = projects_mjtheme_missing.fillna(method='ffill')
print(projects_mjtheme_missing)

#try recreating the column fully instead of using an ffill

     code                 name
458     1  Economic management
1235    1  Economic management
1230    1  Economic management
1229    1  Economic management
1218    1  Economic management
900     1  Economic management
648     1  Economic management
647     1  Economic management
1078    1  Economic management
1206    1  Economic management
1437    1  Economic management
357     1  Economic management
363     1  Economic management
1010    1  Economic management
784     1  Economic management
1024    1  Economic management
88      1  Economic management
1045    1  Economic management
497     1  Economic management
1056    1  Economic management
1057    1  Economic management
1114    1  Economic management
454     1  Economic management
453     1  Economic management
1257    1  Economic management
1260    1  Economic management
1212    1  Economic management
249     1  Economic management
841     1  Economic management
220     1  Economic management
...   ...                  ...
669     

Approach 2: Replace the missing values in the main data frame

In [19]:
#Using projects_mjtheme_name from above, replace the empty strings with NaN, as well as using correct data type
projects_mjtheme_missing = projects_mjtheme_name.replace('',np.nan)
projects_mjtheme_missing.code = projects_mjtheme_missing.code.astype(str)
projects_mjtheme_missing.name = projects_mjtheme_missing.name.astype(str)
#create a function for assigning a name based on a given code
def code_to_name(code): 
    if code == '1' :
        return 'Economic management'
    elif code == '2' :
        return 'Public sector governance'
    elif code == '3' :
        return 'Rule of law'
    elif code == '4' :
        return 'Financial and private sector development'
    elif code == '5' :
        return 'Trade and integration'
    elif code == '6' :
        return 'Social protection and risk management'
    elif code == '7' :
        return 'Social dev/gender/inclusion'
    elif code == '8' :
        return 'Human development'
    elif code == '9' :
        return 'Urban development'
    elif code == '10' :
        return 'Rural development'
    elif code == '11' :
        return 'Environment and natural resources management'

for row in projects_mjtheme_missing.itertuples():
    if pd.isnull(row.name):
        row.name=code_to_name(row.code)
        
projects_mjtheme_missing.describe()

,code,name
count,1499,1499
unique,11,12
top,11,Environment and natural resources management
freq,250,223


Use a Lookup Table and merge

In [61]:
#create lookup table
code_key ={'code' : ['1', '2', '3', '4', '5', '6','7','8','9','10','11'], 
                'name' : ['Economic management','Public sector governance','Rule of law','Financial and private sector development',
                          'Trade and integration','Social protection and risk management','Social dev/gender/inclusion','Human development',
                         'Urban development','Rural development','Environment and natural resources management']}
#merge lookup table with the mjtheme dataframe

pd.merge(projects_mjtheme_name,pd.DataFrame(code_key), on='code')

,code,name_x,name_y
0,8,Human development,Human development
1,8,,Human development
2,8,Human development,Human development
3,8,Human development,Human development
4,8,Human development,Human development
5,8,Human development,Human development
6,8,Human development,Human development
7,8,Human development,Human development
8,8,Human development,Human development
9,8,Human development,Human development
